In [2]:
!pip install -r requirements.txt

  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


You should consider upgrading via the 'C:\Users\ajinf\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [8]:
!pip install --upgrade transformers
#I had to upgrade transformers
!pip install --upgrade trl 

You should consider upgrading via the 'C:\Users\ajinf\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.



  Attempting uninstall: trl
    Found existing installation: trl 0.4.7
    Uninstalling trl-0.4.7:
      Successfully uninstalled trl-0.4.7


You should consider upgrading via the 'C:\Users\ajinf\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [9]:
import torch
import pandas as pd
from tqdm import tqdm
from Tunner import fine_tunner

## Read the data

In [11]:
df = pd.read_csv("Data/train_samples.csv")[:100]
df.head()

,question,answer,source,focus_area
0,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma
1,What causes Glaucoma ?,"Nearly 2.7 million people have glaucoma, a lea...",NIHSeniorHealth,Glaucoma
2,What are the symptoms of Glaucoma ?,Symptoms of Glaucoma Glaucoma can develop in ...,NIHSeniorHealth,Glaucoma
3,What are the treatments for Glaucoma ?,"Although open-angle glaucoma cannot be cured, ...",NIHSeniorHealth,Glaucoma
4,What is (are) Glaucoma ?,Glaucoma is a group of diseases that can damag...,NIHSeniorHealth,Glaucoma


In [12]:
tuner = fine_tunner.Llama_Tuner()

In [13]:
train_data = tuner.to_dataset(list(df['question']), list(df['answer']))

i = 0
for _ in train_data:
  print(_)
  i += 1
  if i >= 10:
    break

{'text': "<s>[INST] What is (are) Glaucoma ? [/INST] Glaucoma is a group of diseases that can damage the eye's optic nerve and result in vision loss and blindness. While glaucoma can strike anyone, the risk is much greater for people over 60. How Glaucoma Develops  There are several different types of glaucoma. Most of these involve the drainage system within the eye. At the front of the eye there is a small space called the anterior chamber. A clear fluid flows through this chamber and bathes and nourishes the nearby tissues. (Watch the video to learn more about glaucoma. To enlarge the video, click the brackets in the lower right-hand corner. To reduce the video, press the Escape (Esc) button on your keyboard.) In glaucoma, for still unknown reasons, the fluid drains too slowly out of the eye. As the fluid builds up, the pressure inside the eye rises. Unless this pressure is controlled, it may cause damage to the optic nerve and other parts of the eye and result in loss of vision. Op

## Model

In [23]:
from huggingface_hub import notebook_login
notebook_login()

In [26]:
model_name = 'google/gemma-2b-it'
save_name = 'gemma-2b'

In [27]:

quant_config = {
    'load_in_4bit': True,
    'bnb_4bit_quant_type': "nf4",
    'bnb_4bit_compute_dtype': getattr(torch, "float16"),
    'bnb_4bit_use_double_quant': False
}

lora_config = {
    'lora_alpha': 16,
    'lora_dropout': 0.1,
    'r': 64,
    'bias': "none",
    'task_type': "CASUAL_LM",
    #use None if you don't want to use target modules
    'target_modules' : [
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ]
}

train_config = {
    'output_dir': "./results",
    'num_train_epochs': 1,
    'per_device_train_batch_size': 4,
    'gradient_accumulation_steps': 1,
    'optim': "paged_adamw_32bit",
    'save_steps': 25,
    'logging_steps': 25,
    'learning_rate': 2e-4,
    'weight_decay': 0.001,
    'fp16': False,
    'bf16': False,
    'max_grad_norm': 0.3,
    'max_steps': -1,
    'warmup_ratio': 0.03,
    'group_by_length': True,
    'lr_scheduler_type': "constant",
    'report_to': "tensorboard",
    'remove_unused_columns': False
}

In [28]:
tuner.load_model(model_name=model_name, quant_config=quant_config, lora_config=lora_config, train_config=train_config)

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like google/gemma-2b-it is not the path to a directory containing a file named config.json.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

In [ ]:
tuner.tune_and_save(train_data, save_name)

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
You are using 8-bit optimizers with a version of `bitsandbytes` < 0.41.1. It is recommended to update your version as a major bug has been fixed in 8-bit optimizers.


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.91 GiB. GPU 0 has a total capacty of 14.75 GiB of which 1.43 GiB is free. Process 22124 has 13.32 GiB memory in use. Of the allocated memory 12.96 GiB is allocated by PyTorch, and 243.02 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
prompt = "What is (are) Glaucoma ?"
tuner.generate_base_text(prompt)

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant

'<s>[INST] What is (are) Glaucoma ? [/INST] The optic nerve is a bundle of mostly normal sized, growing, growing, and growing sized light microscopes. It can also run through a variety of tissues. It has a variety of surgical tools. </s> - Motion Analysis. </s> - Motion Analysis. </s>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</p>  &nbsp;&nbsp;</'

In [ ]:
tuner.generate_text(prompt)

The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PLBartForCausalLM', 'ProphetNetForCausalLM', 'QDQBertLMHeadModel', 'ReformerModelWithLMHead', 'RemBertForCausalLM', 'Rober

'<s>[INST] What is (are) Glaucoma ? [/INST] Open-angle glaucoma is the most common type of glaucoma seen in the eye. It is called a "glaucoma" vision loss. Glaucoma is caused by a combination of a variety of agents, including laser-cut sawing, high-speed scanning technology, and/or high-pitched scanning technology. The eye movement is affected by a variety of factors.  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  Symptoms  '

In [ ]:
test_df = pd.read_csv("./llm_fine_tuning_optimization/Data/test.csv")[:2]
test_dataset = tuner.to_dataset(list(test_df['question']), list(test_df['answer']))

reference = []
for item in test_dataset:
  reference.append(item['text'])

response = []
base_response = []
for ques in tqdm(test_df['question']):
  base_resp = tuner.generate_base_text(ques)
  resp = tuner.generate_text(ques)
  base_response.append(base_resp.strip())
  response.append(resp.strip())

  base_df = pd.DataFrame(base_response, columns=['response'])
  base_df.to_csv(f"./drive/MyDrive/{model_name}.csv", index=False)

  tuned_df = pd.DataFrame(response, columns=['response'])
  tuned_df.to_csv(f"./drive/MyDrive/{model_name}_tuned.csv", index=False)

bleu_score_base = tuner.eval_model(reference, base_response, ['bleu'])
print(f"The bleu score for base model is {bleu_score_base}")

bleu_score_tuned = tuner.eval_model(reference, response, ['bleu'])
print(f"The bleu score for tuned model is {bleu_score_tuned}")

rouge_score_base = tuner.eval_model(reference, base_response, ['rouge'])
print(f"The rouge score for base model is {rouge_score_base}")

rouge_score_tuned = tuner.eval_model(reference, response, ['rouge'])
print(f"The rouge score for tuned model is {rouge_score_tuned}")

ter_score_base = tuner.eval_model(reference, base_response, ['ter'])
print(f"The ter score for base model is {ter_score_base}")

ter_score_tuned = tuner.eval_model(reference, response, ['ter'])
print(f"The ter score for tuned model is {ter_score_tuned}")

  0%|          | 0/2 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:61: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The model 'PeftModel' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', '

The bleu score for base model is (0.033528395384803164,)
The bleu score for tuned model is (0.041333114185783726,)
The rouge score for base model is (0.050997782705099776,)
The rouge score for tuned model is (0.0598669623059867,)
The ter score for base model is (0.8989459815546772,)
The ter score for tuned model is (0.8992753623188405,)
